<a href="https://colab.research.google.com/github/RandomOgre101/dl-project/blob/main/DL%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [4]:
path = './'

apple_pie_train = glob.glob(path + 'train/apple_pie/*')
apple_pie_test = glob.glob(path + 'test/apple_pie/*')

cannoli_train = glob.glob(path + 'train/cannoli/*')
cannoli_test = glob.glob(path + 'test/cannoli/*')

ramen_train = glob.glob(path + 'train/ramen/*')
ramen_test = glob.glob(path + 'test/ramen/*')

In [5]:
print("Analysis by class: \n\t\tTrain\tTest")
print(f"Apple Pie\t{len(apple_pie_train)}\t{len(apple_pie_test)}")
print(f"Cannoli\t\t{len(cannoli_train)}\t{len(cannoli_test)}")
print(f"Ramen\t\t{len(ramen_train)}\t{len(ramen_test)}")

Analysis by class: 
		Train	Test
Apple Pie	750	250
Cannoli		750	250
Ramen		750	250


In [6]:
class_to_category = {
     0 : 'apple pie',
     1 : 'cannoli',
     2 : 'ramen'
}

In [7]:
train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 shear_range=0.2,
                 zoom_range=0.2,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        path + 'train/',
        target_size=(224,224),
        class_mode='categorical',
        batch_size=32)

test_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = test_datagen.flow_from_directory(
        path + 'test/',
        target_size=(224,224),
        class_mode='categorical',
        batch_size=32)

Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


In [8]:
def create_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    predictions = Dense(3, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

def ensemble(models, model_input):
    outputs = [model(model_input) for model in models]
    avg = tf.keras.layers.average(outputs)
    model = Model(inputs=model_input, outputs=avg, name='ensemble')
    return model

In [9]:
base_model1 = VGG16(weights='imagenet', include_top=False)
model1 = create_model(base_model1)

base_model2 = ResNet50(weights='imagenet', include_top=False)
model2 = create_model(base_model2)

base_model3 = DenseNet121(weights='imagenet', include_top=False)
model3 = create_model(base_model3)

29084464/29084464 [==============================] - 2s 0us/step


In [10]:
model1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model3.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model1.fit(train_generator, validation_data=validation_generator, epochs=10)
model2.fit(train_generator, validation_data=validation_generator, epochs=10)
model3.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10
71/71 [==============================] - 80s 784ms/step - loss: 1.1455 - accuracy: 0.6040 - val_loss: 0.8567 - val_accuracy: 0.7293
Epoch 2/10
71/71 [==============================] - 48s 680ms/step - loss: 0.7562 - accuracy: 0.8004 - val_loss: 0.6265 - val_accuracy: 0.8467
Epoch 3/10
71/71 [==============================] - 47s 664ms/step - loss: 0.5909 - accuracy: 0.8680 - val_loss: 0.5619 - val_accuracy: 0.8680
Epoch 4/10
71/71 [==============================] - 46s 637ms/step - loss: 0.5557 - accuracy: 0.8724 - val_loss: 0.5948 - val_accuracy: 0.8453
Epoch 5/10
71/71 [==============================] - 45s 621ms/step - loss: 0.4445 - accuracy: 0.9058 - val_loss: 0.5065 - val_accuracy: 0.8787
Epoch 6/10
71/71 [==============================] - 46s 644ms/step - loss: 0.4168 - accuracy: 0.9107 - val_loss: 0.6280 - val_accuracy: 0.8400
Epoch 7/10
71/71 [==============================] - 46s 646ms/step - loss: 0.3325 - accuracy: 0.9360 - val_loss: 0.4834 - val_accuracy: 0.8827

In [12]:
models = [model1, model2, model3]
model_input = tf.keras.layers.Input(shape=(224, 224, 3))
ensemble_model = ensemble(models, model_input)

In [13]:
model1.save_weights('model1_weights2.h5')
model2.save_weights('model2_weights2.h5')
model3.save_weights('model3_weights2.h5')

In [14]:
ensemble_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
ensemble_model.evaluate(validation_generator)

24/24 [==============================] - 12s 311ms/step - loss: 0.9370 - accuracy: 0.9480


[0.9370113015174866, 0.9480000138282776]

In [41]:
from keras.preprocessing import image

image_path = 'cannoli.jpg'
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

In [42]:
prediction = ensemble_model.predict(img)
p1 = model1.predict(img)
p2 = model2.predict(img)
p3 = model2.predict(img)

1/1 [==============================] - 0s 25ms/step


In [43]:
prediction, p1, p2 ,p3

(array([[3.3333334e-01, 6.6666669e-01, 1.2032221e-22]], dtype=float32),
 array([[0., 1., 0.]], dtype=float32),
 array([[0., 1., 0.]], dtype=float32),
 array([[0., 1., 0.]], dtype=float32))

In [44]:
print(class_to_category[int(np.argmax(prediction, axis=1))])
print(class_to_category[int(np.argmax(p1, axis=1))])
print(class_to_category[int(np.argmax(p2, axis=1))])
print(class_to_category[int(np.argmax(p3, axis=1))])

cannoli
cannoli
cannoli
cannoli
